In [1]:
from Task1 import Task1

In [2]:
a61b005_df = Task1().data
len(a61b005_df)

103560

In [3]:
a61b005_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103560 entries, 0 to 103559
Data columns (total 10 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  103560 non-null  int64 
 1   公告(開)號      103560 non-null  object
 2   KindCode    103560 non-null  object
 3   公告(開)日      103560 non-null  object
 4   名稱          103560 non-null  object
 5   申請日         103560 non-null  object
 6   摘要          102784 non-null  object
 7   申請範圍        102784 non-null  object
 8   專利權人        96694 non-null   object
 9   IPC         102784 non-null  object
dtypes: int64(1), object(9)
memory usage: 7.9+ MB


In [7]:
a61b005_df.head()

,Unnamed: 0,公告(開)號,KindCode,公告(開)日,名稱,申請日,摘要,申請範圍,專利權人,IPC
0,0,US05582169,-,1996-12-10,Method and apparatus for measuring concentrati...,1994-06-08,Optical pulses emitted from an optical pulse s...,What is claimed is:\n1.\n1. A method of measur...,Hamamatsu Photonics K.K.,A61B005/00
1,1,US05582183,-,1996-12-10,Apparatus for achilles tendon reflex testing,1995-07-28,"An apparatus for simultaneously causing, measu...",The embodiments of the invention in which an e...,NaN,A61B005/103
2,2,US05543710,-,1996-08-06,NMR conformal solenoidal coil,1994-02-22,A NMR conformal solenoidal coil includes a gen...,I claim:\n1.\n1. A NMR conformal solenoidal co...,Board of Regents of Univ. of NE,G01V003/00\nG01V003/14\nA61B005/055
3,3,US05507299,-,1996-04-16,Multi-vial blood collection system,1994-06-13,A blood collection system is provided for coll...,I claim:\n1.\n1. A blood collection system for...,NaN,A61B005/00
4,4,US05582168,-,1996-12-10,Apparatus and methods for measuring characteri...,1993-01-22,Apparatus and methods for characterizing biolo...,We claim:\n1.\n1. An apparatus for determining...,Georgia Tech Research Corp.,A61B005/00


In [16]:
# df = a61b005_df[a61b005_df['專利權人'].notna()]
# len(df)

df = a61b005_df.fillna('')
len(df)

103560

In [19]:
counts = df.value_counts(subset='專利權人').reset_index().rename({0:'專利數量'},axis=1)

In [20]:
counts

,專利權人,專利數量
0,-,9372
1,,6866
2,"Medtronic, Inc.",1953
3,"Cardiac Pacemakers, Inc.",1453
4,Siemens Aktiengesellschaft,1271
...,...,...
17947,ELC MANAGEMENT LLC,1
17948,OLYMPUS CORPORATION\nBRAIN FUNCTIONS LABORATOR...,1
17949,OLYMPUS CORPORATION\nOLYMPUS MEDICAL SYSTEMS C...,1
17950,ELBIT SYSTEMS LAND AND C4I LTD,1
